<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Closed_Water_Air.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [1]:
from dotmap import DotMap
import pandas as pd

import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range, todf
from tools.trust_ncg import minimize
import tools.che as che

In [2]:
R=8.314 # J/(mol K)
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])

def model(c):
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    W_n_vap = Pw * V_vap / (R * c.T)
    P = (c.air_n + W_n_vap) * R * c.T / V_vap
    return (s.P_f - P)**2

s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(360,300,600)


vsc=VSC(v,s)
model_f = vsc.transform(model)
model_f(vsc.x)

/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


DeviceArray(2.89480362e+08, dtype=float64)

In [3]:
modelf = jax.jit(vsc.transform(model))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

289480362.365714
103475350.84797531
2082044.4158440402
1304.0337137931747
0.0005540264316586775
1.275157280114514e-16
1.275157280114514e-16
1.275157280114514e-16
1.275157280114514e-16
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [4]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


{'T': DeviceArray([365.71923532], dtype=float64)}


In [5]:
def model2(c):
    r=DotMap()
    Pw = p.Pvap(c.T)[3]
    V_vap = c.V_tot - c.Vw_i
    r.W_n_vap = Pw * V_vap / (R * c.T)
    P = (s.air_n + r.W_n_vap) * R * c.T / V_vap
    return ((s.P_f - P)/s.P_f)**2 + (r.W_n_vap - 0.3)**2, r

In [6]:
s=DotMap()
s.air_y = jnp.array([0.78, 0.21, 0.01])
s.W_tot = 1. # 1 kg
s.V_tot = 0.01 # 10 Liters
s.P_i = 1e5 # Pa air pressure
s.P_f = 2e5 # Pa final pressure
s.q = 2000. # watts
s.T_i = 298.

s.Vw_i = s.W_tot/p.rhol(s.T_i)[3]
s.Vair_i = s.V_tot - s.Vw_i
s.air_n = s.P_i * s.Vair_i / (R * s.T_i)

v=DotMap()
v.T = Range(350, 300, 400)
v.V_tot = Range(0.015, 0., 0.03)

vsc=VSC(v,s)

In [7]:
modelf = jax.jit(vsc.transform(model2))
x_min, f = minimize(modelf, vsc.x, verbosity=1)

0.18134390814615023
0.015435531886651324
0.015435531886651324
0.0006605095790480441
1.1954861545456219e-05
1.1954861545456219e-05
1.1954861545456219e-05
1.2919686113340432e-09
1.2919686113340432e-09
1.2919686113340432e-09
1.2919686113340432e-09
1.671341554847893e-16


In [8]:
v_sol = vsc.xtov(x_min)
v_sol.pprint()


{'T': DeviceArray([370.00563789], dtype=float64),
 'V_tot': DeviceArray([0.01120357], dtype=float64)}


In [9]:
vsc.report(model2, x_min)

vector1 vector3            
                  1       1     2     3
W_n_vap         0.3                    
P_f          200000                    
P_i          100000                    
T           370.006                    
T_i             298                    
V_tot     0.0112036                    
Vair_i   0.00899501                    
Vw_i     0.00100499                    
W_tot             1                    
air_n      0.363057                    
air_y                  0.78  0.21  0.01
q              2000

vector3                 vector1
             1     2     3           1
air_y     0.78  0.21  0.01            
W_tot                                1
V_tot                             0.01
P_i                             100000
P_f                             200000
q                                 2000
T_i                                298
Vw_i                        0.00100499
Vair_i                      0.00899501
air_n                         0.363057